In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import textwrap

In [2]:
start_date = datetime.now() - pd.DateOffset(years=5)
end_date= datetime.now()

df = yf.download('TSLA', start=start_date, end=end_date, progress=False)
df.head(5) # Only show first 5 rows

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-09-11,15.825333,16.544666,15.733333,16.473333,16.473333,150642000
2019-09-12,16.513332,16.900000,16.293333,16.391333,16.391333,128718000
2019-09-13,16.464001,16.563334,16.324667,16.346666,16.346666,79696500
2019-09-16,16.400000,16.495333,16.077999,16.187332,16.187332,70921500
2019-09-17,16.164667,16.373333,16.024668,16.319332,16.319332,57981000


In [3]:
def extract_data_from_ticker(ticker_symbol):
  ticker=yf.Ticker(ticker_symbol)
  ticker_info = ticker.info

  data_to_include = ['shortName', 'symbol', 'country', 'phone', 'website', 'fullTimeEmployees', 'marketCap', 'totalRevenue', 'ebitda', 'profitMargins', 'earningsGrowth', 'revenueGrowth', 'lastFiscalYearEnd', 'nextFiscalYearEnd', 'currency']

  filtered_data = [(k, ticker_info.get(k, None)) for k in data_to_include]

  df = pd.DataFrame(filtered_data, columns=['Key', 'Value'])

  df.loc[df['Key'] == 'marketCap', 'Value'] = (df.loc[df['Key'] == 'marketCap', 'Value'] / 1_000_000_000) # Convert marketCap to billion
  df.loc[df['Key'] == 'marketCap', 'Key'] = 'marketCap (bn $)'

  df.loc[df['Key'] == 'totalRevenue', 'Value'] = (df.loc[df['Key'] == 'totalRevenue', 'Value'] / 1_000_000) # Convert totalRevenue to million
  df.loc[df['Key'] == 'totalRevenue', 'Key'] = 'totalRevenue (m $)'

  df.loc[df['Key'] == 'ebitda', 'Value'] = (df.loc[df['Key'] == 'ebitda', 'Value'] / 1_000_000) # Convert ebitda to million
  df.loc[df['Key'] == 'ebitda', 'Key'] = 'ebitda (m $)'

  df.loc[df['Key'] == 'lastFiscalYearEnd', 'Value'] = pd.to_datetime(df.loc[df['Key'] == 'lastFiscalYearEnd', 'Value'], unit='s', errors='coerce')
  df.loc[df['Key'] == 'nextFiscalYearEnd', 'Value'] = pd.to_datetime(df.loc[df['Key'] == 'nextFiscalYearEnd', 'Value'], unit='s', errors='coerce')

  return df

extract_data_from_ticker('MSFT')

,Key,Value
0,shortName,Microsoft Corporation
1,symbol,MSFT
2,country,United States
3,phone,425 882 8080
4,website,https://www.microsoft.com
5,fullTimeEmployees,228000
6,marketCap (bn $),3126.039151
7,totalRevenue (m $),245122.00704
8,ebitda (m $),129433.001984
9,profitMargins,0.35956


In [6]:
def get_company_summary(ticker_symbol, width=120):
    ticker = yf.Ticker(ticker_symbol)
    ticker_info = ticker.info

    data_to_include = ['longBusinessSummary']

    filtered_data = [(k, ticker_info.get(k, None)) for k in data_to_include]

    df = pd.DataFrame(filtered_data, columns=['Key', 'Value'])

    business_summary = df[df['Key'] == 'longBusinessSummary']['Value'].iloc[0]

    # Wrap the text to the specified width
    wrapped_summary = textwrap.fill(business_summary, width=width)

    print(wrapped_summary)

get_company_summary('AAPL')

Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories
worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-
purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and
HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store
that allow customers to discover and download applications and digital content, such as books, music, video, games, and
podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple
Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand
radio stations; Apple News+, a subscription news and magazine service; Apple TV+, which offers exclusive original
content; Apple Card, a co-branded credit card; and Apple P

In [5]:
def plot_share_price(ticker_symbol):
  df = yf.download(ticker_symbol, start=start_date, end=end_date)
  dfc = extract_data_from_ticker(ticker_symbol)
  currency = dfc['Value'][14]
  fig = go.Figure()

  fig.add_trace(go.Scatter(x=df.index, y=df['Close'], mode='lines', name='Share Price'))

  fig.update_layout(title=f'{ticker_symbol} Share Price (5Y)',
                    xaxis_title='Date',
                    yaxis_title=f'Share Price ({currency})',
                    showlegend=True)

  fig.show()

plot_share_price('NVDA')

[*********************100%***********************]  1 of 1 completed
